# 1.1: Pull tweets- #campfire

Pull Old tweets based hashtag and timestamp.  
Please add keys, label and number of twitter based on below comments accordingly.

In [1]:
from TwitterAPI import TwitterAPI
import json

import pandas as pd
import time

In [2]:
# put twitter keys here
consumer_key=''
consumer_secret=''
access_token_key=''
access_token_secret=''

In [8]:
# put params here
# crateria example:'lang:en point_radius:[-95.36 29.76 25mi] (trapped OR rescue OR sos OR flood OR drowning)'

# pull tweets about campfire
params = {
    'query':'lang:en (#campfire)',
    'fromDate':'201811080000',
    'toDate':'201811090000'
}

In [11]:
# functions used to pull call API multiple times

api = TwitterAPI(consumer_key=consumer_key,
                 consumer_secret=consumer_secret,
                 access_token_key=access_token_key,
                 access_token_secret=access_token_secret)

def _get_tweet_count(found_tweets):
    return len(set(tweet['id'] for tweet in found_tweets))

def search(api, params, next_key=None):
    PRODUCT = 'fullarchive'
    LABEL = '' # This is specific to your application
               # i.e. whatever label you set for your Dev environment, and is case sensitive
    if next_key:
        params['next']= next_key
    r = api.request('tweets/search/%s/:%s' % (PRODUCT, LABEL), params)
    print(r.json().keys())
    try:
        results = r.json()['results']
    except:
        print(r.json())
        return None, None
    try:
        next_new = r.json()['next']
    except:
        return results, None
    return results, next_new

def search_tweets(api, params, counts):
#     results = []
    global results
    MAX_COUNTER = 50 # number of times running api
    query_counter = 0
    tweet_counter = 0
    next_key = ''
    while len(results) < counts and query_counter <= MAX_COUNTER:
        result, next_key = search(api, params, next_key)
        print('got %s results' % str(len(result)))
        print('Next: %s' % next_key)
        results.extend(result)
        query_counter += 1
        if _get_tweet_count(results) == tweet_counter:
            break
        if next_key == None:
            break
        tweet_counter = _get_tweet_count(results)
        time.sleep(2)
    return results


In [12]:
# use above functions
# 4000 is the number of the tweets we gonna get
results = []
test_results = search_tweets(api, params, 1000)

dict_keys(['results', 'next', 'requestParameters'])
got 100 results
Next: eyJhdXRoZW50aWNpdHkiOiJkZTVlYjM0Nzk4MTc0ZjAyOTRmOTcxYjc4NWEzNTY0YjZjODViMDQ3YTk0YjE4NDZhZjg2NjEwYzA0MjVmYTQzIiwiZnJvbURhdGUiOiIyMDE4MTEwODAwMDAiLCJ0b0RhdGUiOiIyMDE4MTEwOTAwMDAiLCJuZXh0IjoiMjAxODExMDkwMDAwMDAtMTA2MDY4MzMwNTE5OTI0MzI2NS0wIn0=
dict_keys(['results', 'next', 'requestParameters'])
got 99 results
Next: eyJhdXRoZW50aWNpdHkiOiJlM2VjOThkNWEyYzMzYzI5OGYyZjE2NTRmYjU0OWJlZGZhMzc4NGRkZmI5NjVhMzI1MGE1NDFlMTE4ZGY5MDQ4IiwiZnJvbURhdGUiOiIyMDE4MTEwODAwMDAiLCJ0b0RhdGUiOiIyMDE4MTEwOTAwMDAiLCJuZXh0IjoiMjAxODExMDkwMDAwMDAtMTA2MDY4MzIxNDY2NTExMzYwMC0wIn0=
dict_keys(['results', 'next', 'requestParameters'])
got 100 results
Next: eyJhdXRoZW50aWNpdHkiOiJmNzI5ZjRhNzVmYWNjZjE1MDJhZWE3MjFlYjYzOGRlY2IzMWVlNTg1NmMxOTkyNmU2YTgyODZiYTU1ZjEzYjk4IiwiZnJvbURhdGUiOiIyMDE4MTEwODAwMDAiLCJ0b0RhdGUiOiIyMDE4MTEwOTAwMDAiLCJuZXh0IjoiMjAxODExMDkwMDAwMDAtMTA2MDY4MzE0MTM2OTc0MTMxMi0wIn0=
dict_keys(['results', 'next', 'requestParameters'])
got 

In [14]:
# save test_results in json format
with open('../data/campfire_results_2.json', 'w') as f:
    json.dump(test_results, f)

In [15]:
# Info pulled from tweets
USEFUL_INFO_KEYS = ['created_at', 'text', 'longitute', 'latitute', 'retweet_count', 'favorite_count', 'hashtags']

def extract_tweets(tweets):
    result = []
    for tweet in tweets:
        info = {}
        for key in USEFUL_INFO_KEYS:
            if key == 'hashtags':
                info[key] = [i['text'] for i in tweet['entities']['hashtags']]
            elif key == 'longitute':
                try:
                    info[key] = tweet['coordinates']['coordinates'][0]
                except TypeError:
                    pass
            elif key == 'latitute':
                try:
                    info[key] = tweet['coordinates']['coordinates'][1]
                except TypeError:
                    pass
            else:
                info[key] = tweet[key]
        result.append(info)
    return result

In [16]:
# convert results into dataframe
extracted_test = extract_tweets(test_results)
df_campfire = pd.DataFrame(extracted_test)

In [18]:
# save dataframe as csv
df_campfire.to_csv('../data/campfire_data_2.csv',index=False)

In [19]:
df_campfire.shape

(1098, 5)